In [27]:
#bibliotecas
import gurobipy as gp
from gurobipy import GRB

In [28]:
# Tamanho do problema
n = 5

In [29]:
# Conjunto
nodes = range(n)

In [30]:
# Demandas
b = [-5, 10, 0, -2, -3]

In [31]:
# Matriz de custo
cost = {
    (0, 1): 2,
    (0, 2): 5,
    (1, 2): 3,
    (2, 3): 1,
    (2, 4): 2,
    (3, 0): 0,
    (3, 4): 2,
    (4, 1): 4 }

In [32]:
# Matriz de capacidade
arcs, cap = gp.multidict({
    (0, 1): 10,
    (0, 2): 10,
    (1, 2): 10,
    (2, 3): 10,
    (2, 5): 10,
    (3, 0): 10,
    (3, 4): 10,
    (4, 1): 10})

In [33]:
# Criando o modelo
m = gp.Model('minimum_cost_flow')

Using license file C:\Users\yuri.martins\Anaconda3\lib\site-packages\gurobipy\gurobi.lic
Restricted license - for non-production use only - expires 2022-01-13


In [36]:
# Adicionando variáveis
x = m.addVars(arcs, obj=cost, name='x')

In [37]:
# Função objetivo
m.ModelSense = GRB.MINIMIZE

In [45]:
# Adicioanndo restrições de capacidade do arco
m.addConstrs((x[i,j] <= cap[i, j] for i, j in arcs), "cap")

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>}

In [46]:
# Adicionando restrições de fluxo
m.addConstrs((x.sum(j, '*') - x.sum('*', j) == b[j] for j in nodes), "flow")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [47]:
# Escrevendo o problema
m.write("min_cost_flow.lp")


In [48]:
# Solucionando o problema
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13 rows, 8 columns and 23 nonzeros
Model fingerprint: 0xfb89079d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+01]
Presolve removed 13 rows and 8 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective  4.600000000e+01


In [51]:
# Print da solução
if m.status == GRB.OPTIMAL:
    print('Optimal solution: %g' % m.objVal)
    solution = m.getAttr('x', x)
    print('\nOptimal flows:')
    for i, j in arcs:
        if solution[i, j] > 0:
            print('%s -> %s: %g' % (i, j, solution[i, j]))

Optimal solution: 46

Optimal flows:
1 -> 2: 10
2 -> 3: 10
3 -> 0: 5
3 -> 4: 3
